# Advent of Code 2024

## Day 01

In [1]:
from pathlib import Path
import winsound

def load_file(file_name: str) -> str:
    with Path(f"./{file_name}").open() as file:
        payload = file.read()
    return payload

def finished_signal(duration: int = 1000, freq: int = 1000):
    winsound.Beep(freq, duration)

In [2]:
# 1st half
import numpy as np

payload = load_file("d01.txt")

left = []
right = []

for pair in payload.split("\n"):
    l, r = pair.split("   ")
    left.append(int(l))
    right.append(int(r))

left = np.array(left)
right = np.array(right)

left = np.sort(left)
right = np.sort(right)

total_distance = np.absolute(left - right).sum()
total_distance

2378066

In [3]:
# 2nd half
from collections import Counter

payload = load_file("d01.txt")

left = []
right = []

for pair in payload.split("\n"):
    l, r = pair.split("   ")
    left.append(int(l))
    right.append(int(r))

right = Counter(right)

similarties = []
for l in left:
    r = right[l]
    s = r * l
    similarties.append(s)

sum(similarties)

18934359

## Day 02

In [4]:
# 1st half
payload = load_file("d02.txt")

def check_diffs(diffs: np.array) -> bool:
    absolute = np.absolute(diffs)
    d_min = np.min(absolute)
    d_max = np.max(absolute)

    if d_min >= 1 and d_max <= 3:
        return True
    else:
        return False

def check_signs(signs: np.array) -> bool:
    pos = np.ones(signs.shape, dtype=np.int8)
    neg = pos * -1  
    if (signs == pos).all() or (signs == neg).all():
        return True
    else:
        return False
    
def check_safety(nums) -> bool:
    diffs = np.diff(nums)
    signs = np.sign(diffs)

    check1 = check_diffs(diffs)
    check2 = check_signs(signs)
    return all([check1, check2])

reports = []
for report in payload.split("\n"):
    nums = map(int, report.split())
    nums = np.array(list(nums))
   
    safe = check_safety(nums)
    reports.append(safe)
    
n_safe_report = sum(map(int, reports))
n_safe_report

564

In [5]:
# 2nd half
payload = load_file("d02.txt")

def check_dampener(nums) -> bool:
    
    for ind in range(len(nums)): 
        lev = nums.copy()
        lev = np.delete(lev, ind)

        safe = check_safety(lev)
        if safe:
            break
    
    return safe


reports = []
for report in payload.split("\n"):
    nums = map(int, report.split())
    nums = np.array(list(nums))
   
    safe = check_safety(nums)
    if not safe:
        safe = check_dampener(nums)

    reports.append(safe)
    
n_safe_report = sum(map(int, reports))
n_safe_report

604

## Day 03

In [6]:
# 1st half
import re 
# build and test pattern at https://regex101.com/

payload = load_file("d03.txt")

mul_pattern = r"mul\(\d+\,\d+\)"
mul_matches = re.findall(mul_pattern, payload)

num_pattern = r"\d+"
num_matches = [re.findall(num_pattern, mul) for mul in mul_matches]

products = []
for pair in num_matches:
    a, b = int(pair[0]), int(pair[1])
    products.append(a * b)

sum(products)

188116424

In [7]:
# 2nd half
pattern = r"(mul\(\d+\,\d+\)|do\(\)|don't\(\))"
instructions = re.findall(pattern, payload)

def update_enable(instr: str) -> bool:
    if instr == "do()":
        return True
    else:
        return False
  
# enable at start
enable = True
products = []

for instr in instructions:
    if instr.startswith("d"):
        enable = update_enable(instr=instr)
    elif enable:
        pair = re.findall(num_pattern, instr)
        a, b = int(pair[0]), int(pair[1])
        products.append(a * b)
    else:
        continue

sum(products)

104245808

## Day 04

In [8]:
# 1st half
# find all occurences of XMAS
# horizontal, vertical, diagonal, backwards
from itertools import product

# directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
directions_dict = list(product([-1, 0, 1], repeat=2))
directions_dict = [d for d in directions_dict if d != (0, 0)]

payload = load_file("./d04.txt")
payload = payload.split("\n")

size = len(payload) # assume n by n

s = 0
for y in range(size):
    for x in range(size):
        for dy, dx in directions_dict:
            if 0 <= y + 3 * dy < size and 0 <= x + 3 * dx < size:
                if payload[y][x] == "X" \
                and payload[y+dy][x+dx] == "M" \
                and payload[y + 2 * dy][x + 2 * dx] == 'A' \
                and payload[y + 3 * dy][x + 3 * dx] == 'S':
                    s += 1

print("Result", s)

Result 2514


## Day 05

In [9]:
# 1st half
from collections import defaultdict
payload = load_file("./d05.txt")

def extract_rules_orders(payload: str) -> tuple[dict[int, list[int]], list[int]]:
    rules = defaultdict(list)
    orders = []
    r_sep = "|"
    o_sep = ","

    for line in payload.split():
        if r_sep in line:
            a, b = line.split(r_sep)
            a, b = int(a), int(b)
            rules[a].append(b)
        elif o_sep in line:
            order = line.split(o_sep)
            order = list(map(int, order))
            orders.append(order)
        else: # skips empty lines
            continue

    rules = dict(rules)
    return rules, orders

rules, orders = extract_rules_orders(payload)

def check_page_index(page: int, order: list[int], rule: list[int]) -> bool:
    page_ind = order.index(page)
    other_ind = [page_ind < order.index(r) for r in rule if r in order]
    return all(other_ind)

valid_orders = {}
for ord_ind, order in enumerate(orders):
    all_valid = []
    for page in order:
        # determine if each individual page is in valid position
        rule = rules.get(page)
        if rule is not None:
            other_ind = check_page_index(page, order, rule)
            all_valid.append(other_ind)
        else:
            all_valid.append(True)
    
    # determine if all pages are in valid positions
    valid_orders[ord_ind] = all(all_valid)


def get_middle_item(any_list: list[int]) -> int:
    return any_list[len(any_list)//2]

s = 0
for ind, valid in valid_orders.items():
    if valid:
        order = orders[ind]
        middle = get_middle_item(order)
        s += middle

s

6267

In [10]:
# 2nd half

def extract_rule_pairs(payload: str) -> list[tuple[int, int]]:
    r_rules = []
    r_sep = "|"

    for line in payload.split():
        if r_sep in line:
            a, b = line.split(r_sep)
            a, b = int(a), int(b)
            r_rules.append((a, b))

    return r_rules

def fix_ordering(rules: list[tuple[int, int]], order: list[int]) -> list[int]:
    rule_dict = defaultdict(list)

    for before, after in rules:
        if before in order and after in order:
            rule_dict[after].append(before)
    # we're reversing the rules for a page with rule 
    # then using len() gives us how many pages must come before
    # thus giving the position were the page in question goes 
    # if no rule for a page is found the defaultdict return empty list
    order.sort(key=lambda page: len(rule_dict[page]))
    return order


r_rules = extract_rule_pairs(payload)

s = 0
for ind, valid in valid_orders.items():
    if not valid:
        order = orders[ind]
        fixed_order = fix_ordering(r_rules, order)
        middle = get_middle_item(fixed_order)
        s += middle

s

5184

## Day 06

In [6]:
# 1st half
payload = load_file("./d06.txt").split()

def make_grid(payload: list[str]) -> dict[tuple[int, int], str]:
    size = len(payload)
    grid = {}

    # assume n by n 
    for x in range(size):
        for y in range(size):
            grid[(x, y)] = payload[y][x]

    return grid


def find_start_coords(symbol: str, grid) ->tuple[int, int]:
    for coords, field in grid.items():
        if field == symbol:
            break

    return coords

def within_bounds(coords, bounds) -> bool:
    if bounds[0] <= coords[0] < bounds[1] \
        and bounds[0] <= coords[1] < bounds[1]:
        return True
    else: 
        return False

directions_dict = {
    "^": (0, -1),
    ">": (1, 0),
    "v": (0, 1),
    "<": (-1, 0),
}

def change_direction(curr_dir: str) -> str:
    next_direction = {
        "^": ">",
        ">": "v",
        "v": "<",
        "<": "^",
    }
    return next_direction[curr_dir]



size = len(payload)
floor = "."
crate = "#"
bounds = (0, size)
grid = make_grid(payload)

curr_dir = "^"
curr_coords = find_start_coords(curr_dir, grid)
distinct_positions = [curr_coords]

while True:
    direction = directions_dict[curr_dir]
    next_coords = (
        curr_coords[0] + direction[0],
        curr_coords[1] + direction[1],
    )

    
    if not within_bounds(next_coords, bounds):
        break
    
    if grid[next_coords] == crate:
        # curr_dir = next(directions)
        curr_dir = change_direction(curr_dir)
        continue

    if grid[next_coords] != crate:
        distinct_positions.append(next_coords)
        curr_coords = next_coords


len(set(distinct_positions))

5095

In [12]:
payload = load_file("./d06.txt").split()
obstacle = "O"

def is_infinite_loop(
        grid: dict[tuple[int, int], str],
        start_coords: tuple[int, int],
        start_dir: str,
        bounds: tuple[int, int],
    ) -> bool:

    curr_coords = start_coords
    curr_dir = start_dir
    visited_states = set()

    while True:
        state = (curr_coords, curr_dir)

        if state in visited_states:
            return True
        visited_states.add(state)

        direction = directions_dict[curr_dir]
        next_coords = (
            curr_coords[0] + direction[0],
            curr_coords[1] + direction[1],
        )

        if not within_bounds(next_coords, bounds):
            return False

        if grid[next_coords] in (crate, obstacle):
            curr_dir = change_direction(curr_dir)
        else:
            curr_coords = next_coords 

def find_valid_obstacle_positions(payload: str) -> list:
    size = len(payload)
    bounds = (0, size)
    grid = make_grid(payload)
    start_dir = "^"
    start_coords = find_start_coords(start_dir, grid)
    valid_positions = []

    for x in range(size):
        for y in range(size):
            pos = (x, y)

            if grid[pos] in (start_dir, crate):
                continue

            grid[pos] = obstacle

            if is_infinite_loop(grid, start_coords, start_dir, bounds):
                valid_positions.append(pos)
            
            grid[pos] = floor # reset grid


    return valid_positions
    
                
val_pos_obst = find_valid_obstacle_positions(payload)
len(val_pos_obst)

1933

## Day 07

In [ ]:
# 1st half
# operators + * -> try all combinations
# find all left that are possible from right with operators 
from math import prod
from itertools import product 
OPERATORS = ("+", "*")

OPS_FUNC = {
    "+": sum,
    "*": prod,
}


def permutator(operators: tuple = OPERATORS, repeat: int = 2):
    return list(product(operators, repeat=repeat))


def check_possible(left: int, right: list[int]) -> bool:
    n_ops = len(right) - 1
    poss_operators = permutator(repeat=n_ops)

    for ops in poss_operators:
        new_left = right[0]

        for i in range(n_ops):
            func = OPS_FUNC[ops[i]]
            new_left = func([new_left, right[i+1]])

            if new_left > left:
                continue   
        
        if new_left == left:
            return True

    return new_left == left


def process_step1(payload: str) -> list[int]:

    result = []
    for line in payload.split("\n"):
        left, right = line.split(":")
        left = int(left)
        right = list(map(int, right.split()))
        if check_possible(left, right):
            result.append(left)

    return result


payload = load_file("./d07.txt")
result = process_step1(payload)
sum(result) # 2941973819040

2941973819040

In [ ]:
# 2nd half

def concat(array: list[int]) -> int:
    return int("".join(map(str, array)))

OPERATORS = ("+", "*", "||")

OPS_FUNC = {
    "+": sum,
    "*": prod,
    "||": concat,
}

payload = load_file("./d07.txt")
result = process_step1(payload)
finished_signal()
sum(result) # 249943041417600

249943041417600

## Day 08

In [10]:
from collections import defaultdict
from itertools import combinations

payload = load_file("./d08.txt")
grid = payload.split("\n")
size = len(grid)
BOUNDS = (0, size)
EMPTY = "."

def within_bounds(point, bounds=BOUNDS):
    return bounds[0] <= point[0] < bounds[1] and bounds[0] <= point[1] < bounds[1]

def compute_antinodes(a, b):
    ax, ay = a
    bx, by = b
    
    c = ax - (bx - ax), ay - (by - ay)
    d = bx + (bx - ax), by + (by - ay)

    if within_bounds(c):
        yield c
    if within_bounds(d):
        yield d


def get_antenna_locations(grid):
    size = len(grid)
    all_locs = defaultdict(list)
    for r in range(size):
        for c in range(size):
            if grid[r][c] != EMPTY:
                all_locs[grid[r][c]].append((r, c))
    
    return all_locs

def get_antinode_locations(antenna_locations):
    antinodes = set()
    for locs in antenna_locations.values():
        # get all two-pair combinations between antennas
        for a, b in combinations(locs, r=2):
            for antinode in compute_antinodes(a, b):
                antinodes.add(antinode)
    
    return antinodes


antenna_locations = get_antenna_locations(grid)
antinode_locations = get_antinode_locations(antenna_locations)

print(len(antinode_locations))

426


In [ ]:

def compute_resonators(a, b):
    ax, ay = a
    bx, by = b
    dx, dy = bx - ax, by - ay
    
    resonators = []
    i = 0
    while True:
        r = ax - dx * i, ay - dy * i
        if within_bounds(r):
            resonators.append(r)
        else:
            break
        i += 1
    
    i = 0
    while True:
        r = bx + dx * i, by + dy * i 
        if within_bounds(r):
            resonators.append(r)
        else:
            break
        i += 1

    return resonators

def get_resonator_locations(antenna_locations):
    resonators = set()
    for freq in antenna_locations:
        locs = antenna_locations[freq]
        for a, b in combinations(locs, r=2):
            for resonator in compute_resonators(a, b):
                resonators.add(resonator)

    return resonators


antenna_locations = get_antenna_locations(grid)
resonator_locations = get_resonator_locations(antenna_locations)


print(len(resonator_locations))

1359
